In [10]:
#%pip install agentpy
#%pip install owlready2

In [11]:
from owlready2 import *
# Model design
import agentpy as ap
import numpy as np
# Visualization
import seaborn as sns

In [12]:
onto = get_ontology("wealth_ontology.owl").load()

In [13]:
#Tienda

with onto: 
    class Agent(Thing):
        pass
    class TiendaAgente(Agent):
        pass
    class PedidosEnTienda(Thing):
        pass
    class has_pedidos(FunctionalProperty, ObjectProperty):
        domain = [TiendaAgente]
        range = [PedidosEnTienda]

In [14]:
class Message():
    environment_buffer = []
    def __init__(self,sender=None,receiver=None,performative=None,content=None):
        self.sender = sender
        self.receiver = receiver
        self.performative = performative
        self.content = content
    def __str__(self):
        return f"\n\
        Sender: {self.sender}, \n\
        Receiver: {self.receiver}, \n\
        Performative: {self.performative}, \n\
        Content: {self.content}"
    def send(self):
        """
        The send function is used to send a message to the environment buffer.
        """
        Message.environment_buffer.append(self)


In [15]:
class TiendaAgente(ap.Agent):
    def setup(self):
        self.recordedPedidos = []